In [5]:
# | default_exp article

In [6]:
# | export
from sqlmodel import SQLModel, Field, create_engine
from pydantic import field_validator
from datetime import datetime, date
from typing import Optional, Dict, Any, List
from pathlib import Path

from sqlmodel import Session, select
from typing import List, Optional

In [7]:
# | export
from seo_rat.utils import *

In [8]:
# | export
class Article(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    website_id: int = Field(foreign_key="website.id")
    file_path: str  # Path to .md file
    focus_keyword: str | None = None
    created_at: datetime = Field(default_factory=datetime.now)


/home/kobo/Desktop/seo_rat/.venv/lib/python3.13/site-packages/sqlmodel/main.py:632: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Article, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


In [9]:
# | export
def insert_article(
    session: Session, website_id: int, file_path: str, focus_keyword: str = None
) -> Article:
    """Insert new article"""
    article = Article(
        website_id=website_id, file_path=file_path, focus_keyword=focus_keyword
    )
    session.add(article)
    session.commit()
    session.refresh(article)
    return article


In [10]:
# | export
def get_article_by_id(session: Session, article_id: int) -> Optional[Article]:
    """Get article by ID"""
    return session.get(Article, article_id)


In [11]:
# | export
def get_article_by_path(session: Session, file_path: str) -> Optional[Article]:
    """Get article by file path"""
    statement = select(Article).where(Article.file_path == file_path)
    return session.exec(statement).first()


In [12]:
# | export
def get_articles_by_website(session: Session, website_id: int) -> List[Article]:
    """Get all articles for a website"""
    statement = select(Article).where(Article.website_id == website_id)
    return session.exec(statement).all()


In [13]:
# | export
def update_article_keyword(
    session: Session, article_id: int, focus_keyword: str
) -> Optional[Article]:
    """Update article focus keyword"""
    article = session.get(Article, article_id)
    if article:
        article.focus_keyword = focus_keyword
        session.add(article)
        session.commit()
        session.refresh(article)
    return article


In [14]:
# | export
def delete_article(session: Session, article_id: int) -> bool:
    """Delete article"""
    article = session.get(Article, article_id)
    if article:
        session.delete(article)
        session.commit()
        return True
    return False

In [21]:
# | test
from fastcore.test import test_eq
from sqlmodel import create_engine, Session, SQLModel
from seo_rat.models import Website
from seo_rat.article import Article

# Create in-memory database
engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    # Create website first
    website = Website(url="https://test.com", name="Test Site", lang="en")
    session.add(website)
    session.commit()
    session.refresh(website)

    # Test insert article
    article = insert_article(
        session, website_id=website.id, file_path="/test/article.md"
    )
    test_eq(article.file_path, "/test/article.md")
    # Test get by path
    found = get_article_by_path(session, "/test/article.md")
    test_eq(found.file_path, "/test/article.md")

    # Test update keyword
    updated = update_article_keyword(session, article.id, "new keyword")
    test_eq(updated.focus_keyword, "new keyword")

    # Test delete
    deleted = delete_article(session, article.id)
    test_eq(deleted, True)


In [23]:
found

Article(website_id=1, created_at=datetime.datetime(2026, 1, 16, 3, 16, 8, 807472), file_path='/test/article.md', id=1, focus_keyword='new keyword')